In [ ]:
    "#위에서 제시한 roi 기준을 재동님이 참고해서 추가적으로 수정한 최종코드1\n",
    "\n",
    "import cv2\n",
    "import numpy as np\n",
    "import imutils  # resize1\n",
    "import cv2\n",
    "import numpy as np\n",
    "#신호등\n",
    "def set_blob_param(r_type: str):\n",
    "    # BLOB 필터 생성하기 \n",
    "    params = cv2.SimpleBlobDetector_Params()\n",
    "    \n",
    "    params.minThreshold = 10\n",
    "    params.maxThreshold = 255\n",
    "    params.thresholdStep = 10\n",
    "    params.filterByArea = False\n",
    "    params.filterByColor = False\n",
    "    params.filterByCircularity = True\n",
    "    params.filterByConvexity = False \n",
    "    params.filterByInertia = False \n",
    "    \n",
    "    if r_type == 'color':\n",
    "        # 경계 값 조정\n",
    "        params.minCircularity = 0.8\n",
    "        params.maxCircularity = 1     \n",
    "    elif r_type == 'shape':\n",
    "        params.filterByArea = True \n",
    "        params.minArea = 100\n",
    "        params.maxArea = 250\n",
    "        \n",
    "        params.filterByColor = False\n",
    "#         params.blobColor = 0\n",
    "        \n",
    "        params.minCircularity = 0.4\n",
    "        params.maxCircularity = 0.8\n",
    "    \n",
    "    return params \n",
    "# 신호등으로 추정되는 영역 검출 \n",
    "def subtract_shape(img):\n",
    "    global k, h, w   # 모폴로지 연산 필터 \n",
    "    \n",
    "    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)\n",
    "    # roi_s = img_gray[int((1/5)*w):int((4/5) * w), int((1/5)*h): int((4/5)*h)]\n",
    "    roi_s = img_gray[int((1/5)*w):int((3/5) * w), int((1/5)*h): int((3/5)*h)]\n",
    "    blur = cv2.GaussianBlur(roi_s, (3, 3), 0)\n",
    "\n",
    "    # ret, th = cv2.threshold(roi_s, 55, 255, cv2.THRESH_BINARY)\n",
    "    _, t_otsu = cv2.threshold(blur, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)\n",
    "\n",
    "#    cv2.imshow('th',t_otsu)\n",
    "    dst_s = cv2.morphologyEx(t_otsu, cv2.MORPH_OPEN, k)   # 모폴로지 열기 연산 \n",
    "    #필터 확인\n",
    "    edge_s = cv2.Laplacian(dst_s, -2)\n",
    "    detector = cv2.SimpleBlobDetector_create(param_s)\n",
    "    \n",
    "    s_keypoints = detector.detect(edge_s)\n",
    "    img_draw_s = cv2.drawKeypoints(roi_s, s_keypoints, None, None, cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) \n",
    "            \n",
    "    global shape_list \n",
    "    \n",
    "    for s_keypoint in s_keypoints:\n",
    "        roi_cx, roi_cy = map(int, s_keypoint.pt)\n",
    "        cx, cy = int(roi_cx + h * (1/5)), int(roi_cy + w * (1/5))\n",
    "#        cx, cy = int(roi_cx + h * (1/13)), int(roi_cy + w * (1/13))\n",
    "        \n",
    "        cv2.putText(img_gray, '!', (cx, cy), cv2.FONT_HERSHEY_TRIPLEX, 1,(255, 255, 255))\n",
    "        size = int(s_keypoint.size)\n",
    "        cx2, cy2 = cx + size // 2, cy + size // 2    \n",
    "        shape_list.append([cx, cx2, cy, cy2])\n",
    "#    cv2.imshow('img_gray',img_gray)\n",
    "    return shape_list  \n",
    "###########\n",
    "# 신호등 색상에 해당되는 영역 검출 \n",
    "def subtract_color(img, color):\n",
    "    global k, w, h   # 모폴로지 연산 필터 \n",
    "    \n",
    "    lower_green = np.array([50, 50, 80]); upper_green = np.array([90, 255, 255])\n",
    "    lower_red = np.array([-10, 30, 30]);  upper_red = np.array([10, 255, 255])\n",
    "    lower_yellow = np.array([11, 50, 50]); upper_yellow = np.array([30, 255, 255])\n",
    "    \n",
    "    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)\n",
    "\n",
    "    roi_c = img_hsv[int((1/5)*w):int((3/5) * w), int((1/5)*h): int((3/5)*h)]\n",
    "    #roi_c = img_hsv[int((1/13)*w):int((2/5) * w), int((1/13)*h): int((2/5)*h)]\n",
    "        \n",
    "    if color == 'red':\n",
    "        mask = cv2.inRange(roi_c, lower_red, upper_red)\n",
    "    elif color == 'green':\n",
    "        mask = cv2.inRange(roi_c, lower_green, upper_green)   \n",
    "    elif color == 'yellow':\n",
    "        mask = cv2.inRange(roi_c, lower_yellow, upper_yellow) \n",
    "        \n",
    "    img_mask = cv2.bitwise_and(roi_c, roi_c, mask=mask)\n",
    "    edge_c = cv2.Laplacian(img_mask, -1)\n",
    "    # dst_c = cv2.morphologyEx(edge_c, cv2.MORPH_OPEN, k) 적용 x (검출률 저하)\n",
    "    detector = cv2.SimpleBlobDetector_create(param_c)\n",
    "    c_keypoints = detector.detect(edge_c)\n",
    "    img_draw_c = cv2.drawKeypoints(roi_c, c_keypoints, None, None, cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)\n",
    "    # cv2.imshow('img_draw_c', img_draw_c)\n",
    "\n",
    "    global color_list \n",
    "    \n",
    "    for c_keypoint in c_keypoints:\n",
    "        roi_cx, roi_cy = map(int, c_keypoint.pt)\n",
    "        cx, cy = int(roi_cx + h * (1/5)), int(roi_cy + w * (1/5))\n",
    "        #cx, cy = int(roi_cx + h * (2/5)), int(roi_cy + w * (2/5))\n",
    "        \n",
    "        cv2.putText(img_hsv, '!', (cx, cy), cv2.FONT_HERSHEY_TRIPLEX, 1,(255, 255, 255))\n",
    "        size = int(c_keypoint.size)\n",
    "        cx2, cy2 = cx + size // 2, cy + size // 2 \n",
    "        color_list.append([cx, cx2, cy, cy2])\n",
    "    \n",
    "    cv2.imshow('img_hsv', img_hsv)        \n",
    "    return color_list\n",
    "                \n",
    "##########################################################################\n",
    "##겹치는 사각형 영역의 꼭지점 수 검출 함수\n",
    "def spot_check(rect1, rect2):\n",
    "    #좀더 좌측 상단에 있는 시작점을 검출해 이것을 기준으로 함\n",
    "    #x좌표를 기준으로 검출\n",
    "    if rect1[0] < rect2[0]:\n",
    "        standard_rect = rect1\n",
    "        other_rect= rect2\n",
    "    else:\n",
    "        standard_rect = rect2\n",
    "        other_rect= rect1\n",
    "    #standard 사각형 기준    \n",
    "                                                           #list(range(1,3)) >>> [1,2]\n",
    "    #range(          x좌표,           x좌표+가로길이(너비)+range는 미만으로 수를 잡으므로 +1 \n",
    "    xrange= range(int(standard_rect[0]),int(standard_rect[0])+int(standard_rect[2])+1) \n",
    "    #range(          y좌표,           y좌표+세로길이(높이)+range는 미만으로 수를 잡으므로 +1 \n",
    "    yrange= range(int(standard_rect[1]),int(standard_rect[1])+int(standard_rect[3])+1)   \n",
    "    \n",
    "    #겹치는지 확인하기 위한 비교대상이 되는 사각형(other_rect) 기준\n",
    "    min_check_x=int(other_rect[0])                #x의 최소값: check_x\n",
    "    max_check_x=int(other_rect[0]+other_rect[2])  #x의 최대값:check_x+check_w\n",
    "    min_check_y=int(other_rect[1])                #y의 최소값: check_y   \n",
    "    max_check_y=int(other_rect[1]+other_rect[3])  #y의 최대값:check_y+check_h \n",
    "    \n",
    "    #검출된 겹치는 꼭지점 개수 초기화\n",
    "    total_points=0\n",
    "    #좌측상단 꼭지점 확인\n",
    "    if min_check_x in xrange and min_check_y in yrange:\n",
    "        total_points+=1\n",
    "    #우측상단 꼭지점 확인\n",
    "    if max_check_x in xrange and min_check_y in yrange:\n",
    "        total_points+=1\n",
    "    #좌측하단 꼭지점 확인\n",
    "    if max_check_y in yrange and min_check_x in xrange:\n",
    "        total_points+=1\n",
    "    #우측하단 꼭지점 확인\n",
    "    if max_check_x in xrange and max_check_y in yrange:\n",
    "        total_points+=1\n",
    "    \n",
    "    return total_points\n",
    "###############################################################################################################################\n",
    "##겹치는 사각형 영역의 꼭지점 수 검출 함수\n",
    "def spot_check(rect1, rect2):\n",
    "    #좀더 좌측 상단에 있는 시작점을 검출해 이것을 기준으로 함\n",
    "    #x좌표를 기준으로 검출\n",
    "    if int(rect1[0]) < int(rect2[0]):\n",
    "        standard_rect = rect1\n",
    "        other_rect= rect2\n",
    "    else:\n",
    "        standard_rect = rect2\n",
    "        other_rect= rect1\n",
    "    #standard 사각형 기준    \n",
    "                                                           #list(range(1,3)) >>> [1,2]\n",
    "    #range(          x좌표,           x좌표+가로길이(너비)+range는 미만으로 수를 잡으므로 +1 \n",
    "    xrange= range(int(standard_rect[0]),int(standard_rect[0])+int(standard_rect[2])+1) \n",
    "    #range(          y좌표,           y좌표+세로길이(높이)+range는 미만으로 수를 잡으므로 +1 \n",
    "    yrange= range(int(standard_rect[1]),int(standard_rect[1])+int(standard_rect[3])+1)   \n",
    "    \n",
    "    #겹치는지 확인하기 위한 비교대상이 되는 사각형(other_rect) 기준\n",
    "    min_check_x=int(other_rect[0])                #x의 최소값: check_x\n",
    "    max_check_x=int(other_rect[0])+int(other_rect[2])  #x의 최대값:check_x+check_w\n",
    "    min_check_y=int(other_rect[1])                #y의 최소값: check_y   \n",
    "    max_check_y=int(other_rect[1])+int(other_rect[3])  #y의 최대값:check_y+check_h \n",
    "    \n",
    "    #검출된 겹치는 꼭지점 개수 초기화\n",
    "    total_points=0\n",
    "    #좌측상단 꼭지점 확인\n",
    "    if min_check_x in xrange and min_check_y in yrange:\n",
    "        total_points+=1\n",
    "    #우측상단 꼭지점 확인\n",
    "    if max_check_x in xrange and min_check_y in yrange:\n",
    "        total_points+=1\n",
    "    #좌측하단 꼭지점 확인\n",
    "    if max_check_y in yrange and min_check_x in xrange:\n",
    "        total_points+=1\n",
    "    #우측하단 꼭지점 확인\n",
    "    if max_check_x in xrange and max_check_y in yrange:\n",
    "        total_points+=1\n",
    "    \n",
    "    return total_points\n",
    "###############################################################################################################################\n",
    "##횡단보도와 사람객체 roi 추적 API\n",
    "#디폴트 에이다 부스트를 이용한 검출 \n",
    "#영상의 길이가 멀어질 수록 잘 검출이 안됨 \n",
    "#사각은 정면보다 특히나 검출이 더 안됨\n",
    "#트랙커 객체 생성자 함수 리스트 \n",
    "trackers = [cv2.TrackerBoosting_create,\n",
    "            cv2.TrackerMIL_create,\n",
    "            cv2.TrackerKCF_create,\n",
    "            cv2.TrackerTLD_create,\n",
    "            cv2.TrackerMedianFlow_create,\n",
    "            #cv2.TrackerGOTURN_create, #버그로 오류 발생 안하는게 좋음\n",
    "            cv2.TrackerCSRT_create,\n",
    "            cv2.TrackerMOSSE_create]\n",
    "trackerIdx = 0  # 트랙커 생성자 함수 선택 인덱스\n",
    "tracker1 = None\n",
    "tracker2 = None\n",
    "isFirst = True\n",
    "video_src = 0 # 비디오 파일과 카메라 선택 \n",
    "video_src = \"./etc/img_data/cropped_traffic2.mp4\"\n",
    "cap = cv2.VideoCapture(video_src)\n",
    "fps = cap.get(cv2.CAP_PROP_FPS) # 프레임 수 구하기\n",
    "delay = int(1000/fps)\n",
    "win_name = 'Tracking APIs'\n",
    "ROIs=0\n",
    "crop_imgs= [] \n",
    "param_c = set_blob_param('color')\n",
    "param_s = set_blob_param('shape')\n",
    "k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))   # 모폴로지 연산 필터 \n",
    "shape_list = []\n",
    "color_list = [] \n",
    "w = 800; h = 800      \n",
    "color_flag = 'green'\n",
    "\n",
    "while cap.isOpened():\n",
    "    global fps, delay, k    \n",
    "    ret, frame = cap.read()\n",
    "    \n",
    "    frame = imutils.resize(frame, width = 800)  # resize1\n",
    "    #frame = cv2.resize(img, (800,800))\n",
    "    \n",
    "    if not ret:\n",
    "        print('Cannot read video file')\n",
    "        break\n",
    "    img_draw = frame.copy()\n",
    "    if tracker1 is None and tracker2 is None: # 트랙커 생성 안된 경우\n",
    "        cv2.putText(img_draw, \"Press the Space to set ROI\", \\\n",
    "            (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2,cv2.LINE_AA)\n",
    "    else:\n",
    "        ok1, bbox1 = tracker1.update(frame)   # 새로운 프레임에서 추적 위치 찾기 \n",
    "        ok2, bbox2 = tracker2.update(frame)   # 새로운 프레임에서 추적 위치 찾기 \n",
    "        (x1,y1,w1,h1) = bbox1\n",
    "        #print('bbox1',bbox1)\n",
    "        (x2,y2,w2,h2) = bbox2\n",
    "        #print('bbox2',bbox2)\n",
    "                \n",
    "        if ok1 and ok2: # 추적 성공\n",
    "            cv2.rectangle(img_draw, (int(x1), int(y1)), (int(x1 + w1), int(y1 + h1)), \\\n",
    "                          (255,0, 0), 2, 1) #img_draw\n",
    "            cv2.rectangle(img_draw, (int(x2), int(y2)), (int(x2 + w2), int(y2 + h2)), \\\n",
    "                          (0,0,255), 2, 1) #img_draw\n",
    "            #1번째 조건부여   \n",
    "            #if사람 객체 roi가 횡단보도 객체roi와 겹친다면 횡단보도 보행자로 인식\n",
    "            if spot_check(bbox1, bbox2)>0: #최소한 꼭지점 하나영역이 겹친다면 \n",
    "                cv2.putText(img_draw, \"STOP!\", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2,cv2.LINE_AA)  \n",
    "                shape_list = subtract_shape(img_draw)\n",
    "                #print('shape_list', shape_list) \n",
    "                color_list = subtract_color(img_draw, color_flag)\n",
    "                #print('color_list', color_list)\n",
    "                tl_flag = False \n",
    "                for a1, a2, b1, b2 in color_list:\n",
    "                    for x1, x2, y1, y2 in shape_list:\n",
    "                        if ((a1 >= x1 and a1 <= x2) and (b1 >= y1 and b1 <= y2)) and tl_flag == False:\n",
    "                            cv2.putText(img_draw, f'{color_flag} Light Detected!', (100, 120), cv2.FONT_HERSHEY_TRIPLEX, 1,(255, 255, 255))\n",
    "                            tl_flag = True \n",
    "            else:\n",
    "                cv2.putText(img_draw, \"\", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2,cv2.LINE_AA)\n",
    "                \n",
    "        else : # 추적 실패\n",
    "            cv2.putText(img_draw, \"Tracking fail.\", (100,80), \\\n",
    "                        cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2,cv2.LINE_AA)\n",
    "    trackerName = tracker1.__class__.__name__\n",
    "    cv2.putText(img_draw, str(trackerIdx) + \":\"+trackerName , (100,20), \\\n",
    "                 cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255),2,cv2.LINE_AA)\n",
    "    cv2.imshow(win_name, img_draw)\n",
    "    key = cv2.waitKey(delay) & 0xff\n",
    "    # 스페이스 바 또는 비디오 파일 최초 실행 \n",
    "    if (video_src != 0 and isFirst):#key == ord(' ') or (video_src != 0 and isFirst): \n",
    "        isFirst = False\n",
    "        ##\n",
    "        hogdef = cv2.HOGDescriptor()\n",
    "        hogdef.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())\n",
    "        hogdaim = cv2.HOGDescriptor((48,96), (16,16), (8,8), (8,8), 9)\n",
    "        hogdaim.setSVMDetector(cv2.HOGDescriptor_getDaimlerPeopleDetector())\n",
    "        cap = cv2.VideoCapture(video_src)\n",
    "        while cap.isOpened():\n",
    "            global ROIs, crop_imgs\n",
    "            ret, img = cap.read()\n",
    "            if img is None:\n",
    "                break\n",
    "            img2 = img.copy()\n",
    "            if ret :\n",
    "                # resize\n",
    "                img = imutils.resize(img, width = 800)  # resize1#counter to save image with different name\n",
    "                # default 디텍터로 보행자 검출\n",
    "                found, _ = hogdef.detectMultiScale(img)\n",
    "                for (x0, y0, w0, h0) in found:\n",
    "                    cv2.rectangle(img, (x0, y0), (x0 + w0, y0 + h0), (0, 255, 255), 2)\n",
    "                cv2.putText(img, 'Detector:%s'%('Default'), (10, 50), cv2.FONT_HERSHEY_DUPLEX,1, (0, 255, 0),1)\n",
    "                cv2.imshow(win_name, img)\n",
    "                # daimler 디텍터로 보행자 검출\n",
    "                key = cv2.waitKey(1)\n",
    "                if key == 27:\n",
    "                    break\n",
    "                if key == 32: # 스페이스\n",
    "                    ROIs = cv2.selectROIs(win_name,img)\n",
    "                    crop_imgs= [] \n",
    "                    for rect in ROIs:\n",
    "                        x1=rect[0]\n",
    "                        y1=rect[1]\n",
    "                        x2=rect[2]\n",
    "                        y2=rect[3]\n",
    "                        crop_imgs.append((x1,y1,x2,y2))\n",
    "                    cv2.waitKey()\n",
    "            else:\n",
    "                break\n",
    "        if len(crop_imgs) != 0:         # 위치 설정 값 있는 경우\n",
    "            tracker1 = trackers[trackerIdx]()    #트랙커 객체 생성 \n",
    "            isInit1 = tracker1.init(frame, crop_imgs[0]) #roi1\n",
    "            tracker2 = trackers[trackerIdx]()    #트랙커 객체 생성\n",
    "            isInit2 = tracker2.init(frame, crop_imgs[1]) #roi2\n",
    "    elif key in range(48, 56): # 0~7 숫자 입력  #근데 안될 수 있는게 있을 수도 있음\n",
    "        trackerIdx = key-48     # 선택한 숫자로 트랙커 인덱스 수정\n",
    "        if bbox1 is not None and bbox2 is not None:\n",
    "            tracker1 = trackers[trackerIdx]() # 선택한 숫자의 트랙커 객체 생성 \n",
    "            isInit1 = tracker1.init(frame, bbox1) # 이전 추적 위치로 추적 위치 초기화\n",
    "            tracker2 = trackers[trackerIdx]() # 선택한 숫자의 트랙커 객체 생성 \n",
    "            isInit2 = tracker2.init(frame, bbox2) # 이전 추적 위치로 추적 위치 초기화\n",
    "    elif key == 27 : \n",
    "        break\n",
    "else:\n",
    "    print( \"Could not open video\")\n",
    "cap.release()\n",
    "cv2.destroyAllWindows()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
